# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from keras.layers.noise import GaussianNoise

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


In [3]:
#Create a sequential model
model = Sequential()

layer = Dense(784, input_shape=(20,), activation='relu', kernel_initializer='glorot_normal', name='in')
model.add(layer)
layer = Dense(192, activation='relu', kernel_initializer='glorot_normal')
model.add(layer)
    


# Define different types of Keras models

In [5]:
l2_lambda_regularization = 0.2
l1_lambda_regularization = 0.10
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    
    
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [6]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [7]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [8]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [10]:
num_features = len(selected_features)

shape = (len(selected_features)*window_size,)
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [11]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [12]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.7)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.7
Loading data from file and computing dataframes
cv number
[18, 33, 47, 49, 53, 99, 58, 80, 24, 67, 45, 60, 23, 63, 68, 1, 88, 4, 91, 39, 70, 56, 100, 46, 98, 5, 83, 94, 86, 71, 43, 77, 87, 27, 25, 12, 17, 10, 65, 11, 7, 96, 48, 81, 6, 69, 92, 35, 16, 97, 57, 15, 29, 40, 38, 13, 93, 59, 44, 20, 3, 84, 37, 30, 78, 31, 9, 32, 74, 75]
Printing shapes

Training data (X, y)
(5426, 336)
(5426, 1)
Cross-Validation data (X, y)
(70, 336)
(70, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.53424658 -0.34595701 -0.73330414 ... -0.6         0.632
   0.4185254 ]
 [-0.58219178 -0.18167861 -0.65228815 ... -0.6         0.328
   0.69795539]
 [-0.76712329 -0.11668373 -0.39697832 ... -0.6         0.28
   0.63847584]
 [-0.67808219 -0.33009212 -0.5231005  ... -0.4         0.28
   0.50960347]
 [-0.64383562 -0.59211873 -0.24107729 ... -0.8         0.264
 

# Train the model and test some Tunable Model functionalities

In [14]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')

shape = (nFeatures*tModel.data_handler.sequence_length,)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

now = datetime.now()
tensorboard = TrainValTensorBoard(log_dir="./logs2/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)
#,tensorboard = tensorboard
tModel.epochs = 200
tModel.train_model(learningRate_scheduler=lrate, verbose=0,tensorboard = tensorboard)




training with cv
Logs: dict_items([('val_loss', 13046.5400390625), ('val_mean_squared_error', 12985.6005859375), ('loss', 8584.108213059804), ('mean_squared_error', 8523.386443383131), ('lr', 0.001)])
Val_logs: {'loss': 13046.5400390625, 'mean_squared_error': 12985.6005859375}
Logs: dict_items([('val_loss', 12174.7783203125), ('val_mean_squared_error', 12112.12109375), ('loss', 8027.611885337956), ('mean_squared_error', 7966.052783689067), ('lr', 0.001)])
Val_logs: {'loss': 12174.7783203125, 'mean_squared_error': 12112.12109375}
Logs: dict_items([('val_loss', 11058.1435546875), ('val_mean_squared_error', 10992.4521484375), ('loss', 7236.175368919611), ('mean_squared_error', 7172.286657809333), ('lr', 0.001)])
Val_logs: {'loss': 11058.1435546875, 'mean_squared_error': 10992.4521484375}
Logs: dict_items([('val_loss', 9706.845703125), ('val_mean_squared_error', 9636.724609375), ('loss', 6276.918887179495), ('mean_squared_error', 6209.450820017336), ('lr', 0.001)])
Val_logs: {'loss': 9706.

Logs: dict_items([('val_loss', 2142.629638671875), ('val_mean_squared_error', 2040.2742919921875), ('loss', 399.61475543012665), ('mean_squared_error', 297.3679491796047), ('lr', 0.001)])
Val_logs: {'loss': 2142.629638671875, 'mean_squared_error': 2040.2742919921875}
Logs: dict_items([('val_loss', 2101.39501953125), ('val_mean_squared_error', 1998.7901611328125), ('loss', 396.1358892185592), ('mean_squared_error', 293.6766009838696), ('lr', 0.001)])
Val_logs: {'loss': 2101.39501953125, 'mean_squared_error': 1998.7901611328125}
Logs: dict_items([('val_loss', 2133.912109375), ('val_mean_squared_error', 2031.1319580078125), ('loss', 392.82844780241044), ('mean_squared_error', 290.1498382489619), ('lr', 0.001)])
Val_logs: {'loss': 2133.912109375, 'mean_squared_error': 2031.1319580078125}
Logs: dict_items([('val_loss', 2112.677978515625), ('val_mean_squared_error', 2009.676513671875), ('loss', 389.40010085986455), ('mean_squared_error', 286.49585302896355), ('lr', 0.001)])
Val_logs: {'loss'

Logs: dict_items([('val_loss', 1983.7852783203125), ('val_mean_squared_error', 1883.986328125), ('loss', 327.8166399856269), ('mean_squared_error', 227.82625438944686), ('lr', 0.001)])
Val_logs: {'loss': 1983.7852783203125, 'mean_squared_error': 1883.986328125}
Logs: dict_items([('val_loss', 1937.0848388671875), ('val_mean_squared_error', 1837.5765380859375), ('loss', 326.5372013664316), ('mean_squared_error', 226.84449170763273), ('lr', 0.001)])
Val_logs: {'loss': 1937.0848388671875, 'mean_squared_error': 1837.5765380859375}
Logs: dict_items([('val_loss', 1973.371337890625), ('val_mean_squared_error', 1874.2305908203125), ('loss', 325.5362500121485), ('mean_squared_error', 226.18041739655385), ('lr', 0.001)])
Val_logs: {'loss': 1973.371337890625, 'mean_squared_error': 1874.2305908203125}
Logs: dict_items([('val_loss', 1982.5841064453125), ('val_mean_squared_error', 1883.7955322265625), ('loss', 325.4946314821901), ('mean_squared_error', 226.47985981014264), ('lr', 0.001)])
Val_logs: {

Logs: dict_items([('val_loss', 1919.841796875), ('val_mean_squared_error', 1826.6058349609375), ('loss', 310.7821999369536), ('mean_squared_error', 217.52268228274275), ('lr', 1e-04)])
Val_logs: {'loss': 1919.841796875, 'mean_squared_error': 1826.6058349609375}
Logs: dict_items([('val_loss', 1918.0897216796875), ('val_mean_squared_error', 1824.8863525390625), ('loss', 310.7614151827752), ('mean_squared_error', 217.53848402147753), ('lr', 1e-04)])
Val_logs: {'loss': 1918.0897216796875, 'mean_squared_error': 1824.8863525390625}
Logs: dict_items([('val_loss', 1920.621826171875), ('val_mean_squared_error', 1827.458984375), ('loss', 310.65121065133445), ('mean_squared_error', 217.45980601551523), ('lr', 1e-04)])
Val_logs: {'loss': 1920.621826171875, 'mean_squared_error': 1827.458984375}
Logs: dict_items([('val_loss', 1918.53173828125), ('val_mean_squared_error', 1825.404296875), ('loss', 310.5811135854134), ('mean_squared_error', 217.43126288419978), ('lr', 1e-04)])
Val_logs: {'loss': 1918.

Logs: dict_items([('val_loss', 1915.16015625), ('val_mean_squared_error', 1823.4781494140625), ('loss', 307.8155170219919), ('mean_squared_error', 216.10318392491016), ('lr', 1e-04)])
Val_logs: {'loss': 1915.16015625, 'mean_squared_error': 1823.4781494140625}
Logs: dict_items([('val_loss', 1914.0548095703125), ('val_mean_squared_error', 1822.413818359375), ('loss', 307.7024647831873), ('mean_squared_error', 216.03611472467634), ('lr', 1e-04)])
Val_logs: {'loss': 1914.0548095703125, 'mean_squared_error': 1822.413818359375}
Logs: dict_items([('val_loss', 1914.6571044921875), ('val_mean_squared_error', 1823.060546875), ('loss', 307.6553171384919), ('mean_squared_error', 216.03712312647042), ('lr', 1e-04)])
Val_logs: {'loss': 1914.6571044921875, 'mean_squared_error': 1823.060546875}
Logs: dict_items([('val_loss', 1909.9320068359375), ('val_mean_squared_error', 1818.3665771484375), ('loss', 307.5586177771091), ('mean_squared_error', 215.97897409171227), ('lr', 1e-04)])
Val_logs: {'loss': 19

Logs: dict_items([('val_loss', 1913.6689453125), ('val_mean_squared_error', 1823.78125), ('loss', 304.44317745063915), ('mean_squared_error', 214.51674796925934), ('lr', 1e-04)])
Val_logs: {'loss': 1913.6689453125, 'mean_squared_error': 1823.78125}
Logs: dict_items([('val_loss', 1901.5123291015625), ('val_mean_squared_error', 1811.644775390625), ('loss', 304.36996342975016), ('mean_squared_error', 214.50111538204723), ('lr', 1e-04)])
Val_logs: {'loss': 1901.5123291015625, 'mean_squared_error': 1811.644775390625}
Logs: dict_items([('val_loss', 1901.9205322265625), ('val_mean_squared_error', 1812.107177734375), ('loss', 304.33301431421773), ('mean_squared_error', 214.49535046193242), ('lr', 1e-04)])
Val_logs: {'loss': 1901.9205322265625, 'mean_squared_error': 1812.107177734375}
Logs: dict_items([('val_loss', 1905.859619140625), ('val_mean_squared_error', 1816.1014404296875), ('loss', 304.2162430366378), ('mean_squared_error', 214.43103247817223), ('lr', 1e-04)])
Val_logs: {'loss': 1905.8

In [15]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_crossVal)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_crossVal)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

70/70 [==============================] - 0s 35us/step
scores
Engine 1, Predicted RUL 81.82805633544922, Rounded RUL 81.0, Real RUL 58.0
Engine 2, Predicted RUL 107.15811920166016, Rounded RUL 107.0, Real RUL 81.0
Engine 3, Predicted RUL 84.05980682373047, Rounded RUL 84.0, Real RUL 57.0
Engine 4, Predicted RUL 5.329617023468018, Rounded RUL 5.0, Real RUL 0.0
Engine 5, Predicted RUL 28.407878875732422, Rounded RUL 28.0, Real RUL 33.0
Engine 6, Predicted RUL 106.09602355957031, Rounded RUL 106.0, Real RUL 122.0
Engine 7, Predicted RUL 125.94464111328125, Rounded RUL 125.0, Real RUL 156.0
Engine 8, Predicted RUL 121.11087036132812, Rounded RUL 121.0, Real RUL 159.0
Engine 9, Predicted RUL 9.373188972473145, Rounded RUL 9.0, Real RUL 19.0
Engine 10, Predicted RUL 84.74337768554688, Rounded RUL 84.0, Real RUL 76.0
Engine 11, Predicted RUL 100.18041229248047, Rounded RUL 100.0, Real RUL 78.0
Engine 12, Predicted RUL 126.4206771850586, Rounded RUL 126.0, Real RUL 123.0
Engine 13, Predicted RU

In [16]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 40us/step
scores
Engine 1, Predicted RUL 116.12946319580078, Rounded RUL 116.0, Real RUL 112.0
Engine 2, Predicted RUL 124.67621612548828, Rounded RUL 124.0, Real RUL 98.0
Engine 3, Predicted RUL 60.862667083740234, Rounded RUL 60.0, Real RUL 69.0
Engine 4, Predicted RUL 87.47703552246094, Rounded RUL 87.0, Real RUL 82.0
Engine 5, Predicted RUL 94.43400573730469, Rounded RUL 94.0, Real RUL 91.0
Engine 6, Predicted RUL 103.78065490722656, Rounded RUL 103.0, Real RUL 93.0
Engine 7, Predicted RUL 108.21924591064453, Rounded RUL 108.0, Real RUL 91.0
Engine 8, Predicted RUL 88.47247314453125, Rounded RUL 88.0, Real RUL 95.0
Engine 9, Predicted RUL 109.77326965332031, Rounded RUL 109.0, Real RUL 111.0
Engine 10, Predicted RUL 97.78631591796875, Rounded RUL 97.0, Real RUL 96.0
Engine 11, Predicted RUL 88.87660217285156, Rounded RUL 88.0, Real RUL 97.0
Engine 12, Predicted RUL 97.50536346435547, Rounded RUL 97.0, Real RUL 124.0
Engine 13, Predicted